In [57]:
import tensorflow as tf

In [58]:
no_steps = 30
n_features = 3
input_shape = (None, no_steps, n_features)
num_units = 40
learning_rate = 0.001

In [59]:
tf.reset_default_graph()

In [60]:
with tf.variable_scope('ForwardtimeI/OLayer'):
    fw_x = tf.placeholder(dtype=tf.float32, shape=input_shape, name='forward_input')
    bw_x = tf.placeholder(dtype=tf.float32, shape=input_shape, name='backward_input')
    y = tf.placeholder(dtype=tf.float32, shape=None,name='y')

In [61]:
with tf.variable_scope('ForwardtimeLSTMLayer'):
    fw_cell = tf.nn.rnn_cell.LSTMCell(activation=tf.nn.elu, dtype=tf.float32,name='forward_cell', num_units=num_units)
    fw_outputs, fw_state = tf.nn.dynamic_rnn(cell=fw_cell,dtype=tf.float32, inputs=fw_x)

In [62]:
with tf.variable_scope('BackwardtimeLSTMLayer'):
    bw_cell = tf.nn.rnn_cell.LSTMCell(activation=tf.nn.elu, dtype=tf.float32, name='backward_cell', num_units=num_units)
    bw_outputs, bw_state = tf.nn.dynamic_rnn(cell=bw_cell, dtype=tf.float32, inputs=bw_x)

In [63]:
with tf.variable_scope('BiDirectionalLSTM'):
    stacked_cell = tf.stack([fw_outputs, bw_outputs], axis=1)
    outputs= tf.layers.dense(activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                             units=1, inputs=stacked_cell)

with tf.variable_scope('loss'):
    ##lets say its mean squared cost function
    loss = tf.reduce_mean(tf.square(y - outputs))
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.99,use_nesterov=True)
    training_op = optimizer.minimize(loss)

### Run this cell in code mode to have an idea on what tf.stack is doing
```
with tf.Session() as sess: 
    print(tf.stack([[1,2,3,4],[5,6,7,8]], axis=1).eval())
    ```

In [64]:
with tf.Session() as sess:
    # ----------------------------------------
    # proceed with training, except when feeding bw_x
    # value of bw_x = value of fw_x [::-1]
    # ----------------------------------------
    pass